In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from models import *

In [3]:
def train(model, optimizer, data, target, num_iters):
    for i in range(num_iters):
        out = model(data)
        loss = F.mse_loss(out, target)
        mea = torch.mean(torch.abs(target - out))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 5000 == 0:
            print("\t{}/{}: loss: {:.3f} - mea: {:.3f}".format(
                i+1, num_iters, loss.item(), mea.item())
            )

## Permutation

In [4]:
# permute the first column with the third

A = torch.from_numpy(np.array([
    [0, 1, -1],
    [3, -1, 1],
    [1, 1, -2],
])).float()

B = torch.from_numpy(np.array([
    [-1, 1, -0],
    [1, -1, 3],
    [-2, 1, 1],
])).float()

P = torch.from_numpy(np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0],
])).float()

assert torch.allclose(torch.matmul(A, P), B)

In [5]:
net = NeuralAccumulatorCell(3, 3)
optim = torch.optim.RMSprop(net.parameters(), lr=1e-4)

train(net, optim, A, B, int(4e4))

print(net.W.data)

	1/40000: loss: 1.444 - mea: 0.927
	5001/40000: loss: 0.077 - mea: 0.177
	10001/40000: loss: 0.000 - mea: 0.007
	15001/40000: loss: 0.000 - mea: 0.000
	20001/40000: loss: 0.000 - mea: 0.000
	25001/40000: loss: 0.000 - mea: 0.000
	30001/40000: loss: 0.000 - mea: 0.000
	35001/40000: loss: 0.000 - mea: 0.000
tensor([[ 0.0000, -0.0000,  1.0000],
        [-0.0001,  1.0000, -0.0001],
        [ 1.0000,  0.0000, -0.0000]])


In [9]:
print("actual: \n{}\n".format(net.W.transpose(0, 1)))
print("expected: \n{}\n".format(P))

actual: 
tensor([[ 0.0000, -0.0001,  1.0000],
        [-0.0000,  1.0000,  0.0000],
        [ 1.0000, -0.0001, -0.0000]])

expected: 
tensor([[ 0.,  0.,  1.],
        [ 0.,  1.,  0.],
        [ 1.,  0.,  0.]])



## Column Scaling

In [10]:
# scale the first column by 5

A = torch.from_numpy(np.array([
    [0, 1, -1],
    [3, -1, 1],
    [1, 1, -2],
])).float()

B = torch.from_numpy(np.array([
    [0, 1, -1],
    [15, -1, 1],
    [5, 1, -2],
])).float()

P = torch.from_numpy(np.array([
    [5, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
])).float()

assert torch.allclose(torch.matmul(A, P), B)

In [11]:
net = NeuralAccumulatorCell(3, 3)
optim = torch.optim.RMSprop(net.parameters(), lr=1e-4)

train(net, optim, A, B, int(9e4))

print(net.W.data)

	1/90000: loss: 28.833 - mea: 3.000
	5001/90000: loss: 21.383 - mea: 2.246
	10001/90000: loss: 16.247 - mea: 1.900
	15001/90000: loss: 11.851 - mea: 1.626
	20001/90000: loss: 8.166 - mea: 1.367
	25001/90000: loss: 5.191 - mea: 1.109
	30001/90000: loss: 2.924 - mea: 0.852
	35001/90000: loss: 1.366 - mea: 0.595
	40001/90000: loss: 0.511 - mea: 0.340
	45001/90000: loss: 0.207 - mea: 0.227
	50001/90000: loss: 0.148 - mea: 0.184
	55001/90000: loss: 0.103 - mea: 0.153
	60001/90000: loss: 0.065 - mea: 0.122
	65001/90000: loss: 0.036 - mea: 0.091
	70001/90000: loss: 0.016 - mea: 0.060
	75001/90000: loss: 0.004 - mea: 0.030
	80001/90000: loss: 0.000 - mea: 0.001
	85001/90000: loss: 0.000 - mea: 0.000
tensor([[ 5.0001e+00, -5.1951e-05,  5.1859e-05],
        [-5.0003e-05,  1.0000e+00, -5.0024e-05],
        [-5.0004e-05,  4.9928e-05,  9.9995e-01]])


In [12]:
print("actual: \n{}\n".format(net.W.transpose(0, 1)))
print("expected: \n{}\n".format(P))

actual: 
tensor([[ 5.0001e+00, -5.0003e-05, -5.0004e-05],
        [-5.1951e-05,  1.0000e+00,  4.9928e-05],
        [ 5.1859e-05, -5.0024e-05,  9.9995e-01]])

expected: 
tensor([[ 5.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]])



## Column Elimination

In [18]:
def basis_vec(k, n):
    """Creates the k'th standard basis vector in R^n."""
    error_msg = "[!] k cannot exceed {}.".format(n)
    assert (k < n), error_msg
    b = np.zeros([n, 1])
    b[k] = 1
    return b

In [19]:
# add -3x the second column to the first => P = (I - (c)(e_k)(e_l.T))

A = torch.from_numpy(np.array([
    [3, 1, -1],
    [3, -1, 1],
    [1, 1, -2],
])).float()

B = torch.from_numpy(np.array([
    [0, 1, -1],
    [6, -1, 1],
    [-2, 1, -2],
])).float()

P = torch.from_numpy(
    np.eye(3) + (-3)*basis_vec(1, 3).dot(basis_vec(0, 3).T)
).float()

assert torch.allclose(torch.matmul(A, P), B)

In [20]:
net = NeuralAccumulatorCell(3, 3)
optim = torch.optim.RMSprop(net.parameters(), lr=1e-4)

train(net, optim, A, B, int(9e4))

print(net.W.data)

	1/90000: loss: 5.444 - mea: 1.667
	5001/90000: loss: 1.509 - mea: 0.679
	10001/90000: loss: 0.226 - mea: 0.247
	15001/90000: loss: 0.084 - mea: 0.165
	20001/90000: loss: 0.038 - mea: 0.111
	25001/90000: loss: 0.010 - mea: 0.057
	30001/90000: loss: 0.000 - mea: 0.005
	35001/90000: loss: 0.000 - mea: 0.000
	40001/90000: loss: 0.000 - mea: 0.000
	45001/90000: loss: 0.000 - mea: 0.000
	50001/90000: loss: 0.000 - mea: 0.000
	55001/90000: loss: 0.000 - mea: 0.000
	60001/90000: loss: 0.000 - mea: 0.000
	65001/90000: loss: 0.000 - mea: 0.000
	70001/90000: loss: 0.000 - mea: 0.000
	75001/90000: loss: 0.000 - mea: 0.000
	80001/90000: loss: 0.000 - mea: 0.000
	85001/90000: loss: 0.000 - mea: 0.000
tensor([[ 9.9995e-01, -3.0001e+00,  5.0084e-05],
        [ 4.9894e-05,  1.0000e+00, -4.9870e-05],
        [ 4.9999e-05,  4.9925e-05,  9.9995e-01]])


In [21]:
print("actual: \n{}\n".format(net.W.transpose(0, 1)))
print("expected: \n{}\n".format(P))

actual: 
tensor([[ 9.9995e-01,  4.9894e-05,  4.9999e-05],
        [-3.0001e+00,  1.0000e+00,  4.9925e-05],
        [ 5.0084e-05, -4.9870e-05,  9.9995e-01]])

expected: 
tensor([[ 1.,  0.,  0.],
        [-3.,  1.,  0.],
        [ 0.,  0.,  1.]])

